In [ ]:
#For the usage of GPU we are utilizing Google Colab, hence the install statements are present here.

In [ ]:
!pip install ollama pandas networkx matplotlib tqdm -q

In [ ]:
!pip install transformers --q

In [ ]:
!pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.5 MB/s eta 0:00:00


In [ ]:
!pip install setfit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.2/75.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.0 MB/s eta 0:00:00


In [ ]:
!pip install wikipedia

In [ ]:
#Imported packages, also turning of WANDB because of a super annoying error message in case you were wondering.
import json
import re
import os
import numpy as np
import wikipedia
import pandas as pd
from tqdm.notebook import tqdm
from collections import defaultdict
from datasets import load_dataset
from datasets import DatasetDict, Dataset
from sklearn.model_selection import train_test_split
from setfit import SetFitModel, Trainer, TrainingArguments, sample_dataset
from transformers import AutoTokenizer, pipeline
from tqdm.notebook import tqdm

os.environ["WANDB_DISABLED"] = "true"

In [ ]:
#We load the english wikipedia dataset from huggingface.
dataset = load_dataset('wikimedia/wikipedia', '20231101.en')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/131k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

train-00000-of-00041.parquet:   0%|          | 0.00/420M [00:00<?, ?B/s]

train-00001-of-00041.parquet:   0%|          | 0.00/351M [00:00<?, ?B/s]

...

train-00038-of-00041.parquet:   0%|          | 0.00/538M [00:00<?, ?B/s]

train-00039-of-00041.parquet:   0%|          | 0.00/465M [00:00<?, ?B/s]

train-00040-of-00041.parquet:   0%|          | 0.00/422M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6407814 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/41 [00:00<?, ?it/s]

In [ ]:
#Having chosen the english dataset of the wikipedia articles it is quite evident that we are not going to be able to work directly with 6.4 million rows of data. Alternatively we could have chosen
#the danish dataset(or another language), however due to concerns of LLMs performance on languages other than english we decided to continue using.
#We also ensure that huggingface has not somehow divided the dataset into splits of train and test yet to ensure we have all the data, and see that all data is labeled train for now.
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 6407814
    })
})


In [ ]:
#In a perfect world with unlimited GPU and time and access to the best computers in the world, maybe we could have started the process of classification on 6.4 million rows of data, however this is not the case.
#Instead to downsize the dataset we opt for a more old-fashioned approach to simply look up the existence of words in the text of each row, to then limit our dataset. Given the scope of the task ahead,
#where we want to find companies that are somehow involved with AI technologies we decide to determine that we only want rows of data where insensitive "company" and the specific text "AI" or insensitive "artifical intelligence"
#is present.
filtered_ds = dataset.filter(lambda x: "company" in x["text"].lower() and
                                       (re.search(r"\bAI\b", x["text"]) or
                                        "artificial intelligence" in x["text"].lower()))

Filter:   0%|          | 0/6407814 [00:00<?, ? examples/s]

In [ ]:
#This has narrowed the 6.4 million rows down to 6892 rows of data.
print((filtered_ds.num_rows, len(filtered_ds.column_names)))

({'train': 6892}, 1)


In [ ]:
#Inspecting a random sample we can see that for example this specific row of data is not relevant to our case. It is not a company nor a technology related to the AI market, but simply an article that somehow
#mentions artifical intelligence. Therefore before we can begin to extract relationships on our data to perform network analysis, we must further ensure that our data is relevant.
filtered_ds['train'][1200]

{'id': '18950885',
 'url': 'https://en.wikipedia.org/wiki/BBC%20Micro',
 'title': 'BBC Micro',
 'text': 'The British Broadcasting Corporation Microcomputer System, or BBC Micro, is a series of Microcomputers designed and built by Acorn Computers Limited in the 1980s for Computer Literacy Project of the BBC. Designed with an emphasis on education, it was notable for its ruggedness, expandability, and the quality of its operating system. An accompanying 1982 television series, The Computer Programme, featuring Chris Serle learning to use the machine, was broadcast on BBC2.\n\nAfter the Literacy Project\'s call for bids for a computer to accompany the TV programmes and literature, Acorn won the contract with the Proton, a successor of its Atom computer prototyped at short notice. Renamed the BBC Micro, the system was adopted by most schools in the United Kingdom, changing Acorn\'s fortunes. It was also successful as a home computer in the UK, despite its high cost. Acorn later employed th

In [ ]:
#Here we opt for the method of few-shot learning implementation using SetFit to further ensure the relevancy of our data. We therefore set out to train the BERT-model using Few-Shot. To do so we need to label
#some wikipedia articles beforehand. Utilizing the package of wikipedia in python we can fetch the article text as long as we provide the title of our relevant articles. Therefore we find a bunch of articles,
#and given that this is few-shot, we dont need all too many to see results. We therefore find about 50 relevant articles and 50 unrelevant articles. Aspirations for relevant articles are company articles
#of actors on the AI market. Irrelevant articles are mostly randomly chosen, however also some have been chosen a bit more knowingly. Having run this process a bunch of times earlier through different
#editions of this code it was apparent that the model struggled with sorting in particular video games and articles of specific people who somehow were related to AI. Therefore examples of these labeled irrelevant
#were chosen knowingly to train the model against them.


#This definition checks the chosen titles and attempts at finding the article text through the wikipedia module. As this is a quite specific task, we also only accept exact matches on the titles we give.
#This means that if it cant find an article related to our given title, then we will not accept a close article. It should instead discontinue that label.
def fetch_and_label_article(title, relevant_titles):
    try:
        search_results = wikipedia.search(title, results=1)

        if search_results and search_results[0].lower() == title.lower():
            page = wikipedia.page(search_results[0])
            text = page.content

            label = "relevant" if title in relevant_titles else "irrelevant"

            return {
                "title": page.title,
                "text": text,
                "label": label
            }
        else:
            print(f"No exact match found for '{title}'. Search returned: {search_results}")

    except wikipedia.exceptions.DisambiguationError as e:
        print(f"Disambiguation error for '{title}': {e.options}")
    except wikipedia.exceptions.PageError:
        print(f"Page not found for '{title}'")
    except Exception as e:
        print(f"An error occurred: {e}")

#our chosen relevant titles of wikipedia articles pages
relevant_titles = [
    "IBM Watsonx", "OpenAI", "Vicarious (company)", "SoundHound", "Google DeepMind", "Clarifai", "Affectiva", "Grammarly", "Adobe Inc.","Luminoso", "Dialogflow", "ChatGPT", "Genpact", "Webtrekk", "USens",
    "Amiga software","Altoros","Siscog", "Eureqa",
    "01.AI", "4Paradigm", "Agility Robotics", "AI21 Labs", "Aleph Alpha", "Anthropic", "Art Recognition", "Artisse AI", "Asteria Aerospace", "Australian Artificial Intelligence Institute", "BabyX", "Baichuan", "Berkshire Grey", "Blue Prism", "Butlr Technologies", "Caper AI", "Colloquis", "Conversable", "CoreWeave", "Covariant", "DataStax", "Dxcover", "Element AI", "Fetch.AI", "Gcore", "Glean Technologies", "ICarbonX", "Inflection AI", "Isomorphic Labs", "Matador Network", "Microsoft", "Mind Foundry", "Moonshot AI", "Onpassive", "Owkin", "Peak", "Perplexity AI", "RAIC Labs", "Respeecher", "Runway", "Spectre AI", "Synthesia", "Tailent Automation Platform", "UiPath", "Undetectable.ai", "Vectra AI", "Writesonic", "XAI", "Zhipu AI",
]

#our chosen total amount of articles and included a bunch of irrelevant articles.
titles = [
    "IBM Watsonx", "OpenAI", "Vicarious (company)", "SoundHound", "Google DeepMind", "Clarifai", "Affectiva", "Grammarly", "Adobe Inc.","Luminoso", "Dialogflow", "ChatGPT", "Genpact", "Webtrekk", "USens",
    "Amiga software","Altoros","Siscog", "Eureqa",
    "01.AI", "4Paradigm", "Agility Robotics", "AI21 Labs", "Aleph Alpha", "Anthropic", "Art Recognition", "Artisse AI", "Asteria Aerospace", "Australian Artificial Intelligence Institute", "BabyX", "Baichuan", "Berkshire Grey", "Blue Prism", "Butlr Technologies", "Caper AI", "Cerence", "Colloquis", "Conversable", "CoreWeave", "Covariant", "DataStax", "Dxcover", "Element AI", "Fetch.AI", "Gcore", "Glean Technologies", "ICarbonX", "Inflection AI", "Isomorphic Labs", "Matador Network", "Microsoft", "Mind Foundry", "Moonshot AI", "NewsRx", "Onpassive", "Owkin", "Peak", "Perplexity AI", "RAIC Labs", "Respeecher", "Runway", "Spectre AI", "Synthesia", "Tailent Automation Platform", "UiPath", "Undetectable.ai", "Vectra AI", "Writesonic", "XAI", "Zhipu AI",
    "History of animation", "Climate change", "Origami", "Applications of artificial intelligence", "Pony.ai", "Music of Genshin Impact", "Sports in the United States",
    "Butterfly", "Artificial intelligence", "Timeline of Amazon Web Services", "Elshan Hajizadeh", "Piano Media",
    "History of the Internet", "Quantum Mechanics", "Mount Everest", "The Great Gatsby", "World War II", "Python (programming language)", "Black Hole", "Albert Einstein", "Photosynthesis",
    "Machine Learning", "Solar System", "The Beatles", "Marie Curie", "Renaissance", "Climate Change", "Blockchain", "Leonardo da Vinci", "Nanotechnology", "Turing Machine",
    "List of acronyms: B", "List of acronyms: N", "List of acronyms: P", "George Arison", "Dean Weber",
    "Urs Hölzle","Phillip Doyce Hester", "Dan Gilbert", "Accelerated Math", "Fenton Communications", "World Kinect Corporation",
    "History of Google","AIM (software)","YourMechanic", "David Silver (computer scientist)","Dean Weber", "Anthony Levandowski",
    "Zango (company)", "Oolite (video game)", "IPad (4th generation)", "IGEL Technology", "Thomas Tull", "Phil Carter",
    "IcedTea", "John E. Laird", "Catherine Havasi", "Eric Xu", "Call of Duty: Modern Warfare 2", "Scribblenauts (video game)",
    "Edge (video game)", "And Who Shall Go to the Ball? And What Shall Go to the Ball?", "Sleeping Dogs (video game)",
    "Dota 2", "Fruit Ninja", "Ninjago (TV series)", "Metro: Last Light", "Rez (video game)", "007 Legends"
]

labeled_examples = []
for title in titles:
    example = fetch_and_label_article(title, relevant_titles)
    if example:
        labeled_examples.append(example)

#We define the looked up text as the text of the data and also the label that we had given as the data we are going to train our model on.
texts = [example['text'] for example in labeled_examples]
labels = [1 if example['label'] == "relevant" else 0 for example in labeled_examples]

#We print the texts so we can inspect if the actual content text that was given our labels are in fact the ones we had wanted. It can be seen that multiple titles failed in the look up, but ultimately there are about 50
#of each labeled class, which we deem enough to continue with our few-shot learning.
for i, (text, label) in enumerate(zip(texts, labels)):
    print(f"Text {i+1} (Label: {label}): {text[:200]}...")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Page not found for 'IBM Watsonx'
Page not found for 'ChatGPT'
Page not found for 'Butlr Technologies'
Page not found for 'CoreWeave'
No exact match found for 'Covariant'. Search returned: ['Covariant derivative']
Page not found for 'Element AI'
Page not found for 'Gcore'
Page not found for 'Onpassive'
An error occurred: An unknown error occured: "Search is currently too busy. Please try again later.". Please report it on GitHub!
No exact match found for 'Synthesia'. Search returned: ['Synthesia (company)']
Page not found for 'UiPath'
Page not found for 'Writesonic'
No exact match found for 'XAI'. Search returned: ['XAI (company)']
An error occurred: An unknown error occured: "Search is currently too busy. Please try again later.". Please report it on GitHub!
Page not found for 'Mount Everest'
Page not found for 'The Great Gatsby'
An error occurred: An unknown error occured: "Search is currently too busy. Please try again later.". Please report it on GitHub!
An error occurred: An unknow

In [ ]:
#We experimented quite a bit with this, but ultimately we ended up providing the model with a prompt in relation to the text, in hopes that the model would perform better, which we then define as the text we want it to find.
def add_prompt(text, prompt="classify the article text as relevant or irrelevant. A relevant article is a business company that also works with ai or artifical intelligence in this article :"):
    return f"{prompt} {text}"
texts_with_prompt = [add_prompt(text) for text in texts]
data_dict = {"text": texts_with_prompt, "label": labels}
full_dataset = Dataset.from_dict(data_dict)

#We split the dataset into first train and test with 80/20 distribution. We ensure that we stratify so that we keep up the ratio of labeled data. We then split half the test data into validation data also making sure to stratify.
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)
valid_texts, test_texts, valid_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels
)

#we define the different splits and combine it into one "dataset1"
train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels})
validation_dataset = Dataset.from_dict({"text": valid_texts, "label": valid_labels})
test_dataset = Dataset.from_dict({"text": test_texts, "label": test_labels})

dataset1 = DatasetDict({
    "train": train_dataset,
    "validation": validation_dataset,
    "test": test_dataset
})

In [ ]:
#Here we implement the few-shot learning. As we expect the LLM to be able perform with only a few samples we decide to use 30 samples.
train_dataset = sample_dataset(dataset1["train"], label_column="label", num_samples=30)
eval_dataset = dataset1["validation"]
test_dataset = dataset1["test"]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#We load a pre-trained BERT tokenizer.
tokenizer_bert = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
#We import the BERT model through Setfit and tell it to label the data relevant or irrelevant. We use BERT mainly for computational concerns.
model_bert = SetFitModel.from_pretrained("bert-base-uncased", labels=['irrelevant', 'relevant'])

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [ ]:
#We tokenize, so we make sure the model can read the text. Here are some choices we make in the tokenization as we choose max_length padding, which means the BERT model will use its maximum tokens which is 512, but by
#having also set truncation=true we allow it to simply cut off once this is reached. This means that potentially some data of the text is lost, but given the earlier examples where we inspected the content of articles it is
#apparent that in relevant articles the data we want has presented itself almost immediately, but on the off-chance a relevant row of data could potentially be lost by this decision. This is again to maintain some limits to
#ensure that the task can be executed with reasonable time and computational power.
def tokenize_function(examples):
    return tokenizer_bert(examples["text"], padding="max_length", truncation=True)
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

In [ ]:
tokenized_train_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 60
})

In [ ]:
#We use Berts standard training arguments.
training_args = TrainingArguments(output_dir="bert_trainer")

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
#We define the data that is to be used that we determined earlier and the settings.
trainer_bert = Trainer(
    model=model_bert,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    metric="accuracy",
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Map:   0%|          | 0/60 [00:00<?, ? examples/s]

In [ ]:
#We train the model
trainer_bert.train()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
***** Running training *****
  Num unique pairs = 1860
  Batch size = 16
  Num epochs = 1


Step,Training Loss
1,0.190500
50,0.191100
100,0.006400


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [ ]:
#We evalutate the model and find that the accuracy is 1.0 or simply 100%. Usually this would mean that it was probably overfitted, but given the low amount of samples characteristic to few-shot this was
#kind of expected one could argue. Instead we proceeded and checked if the rows of data that were labeled were acceptable and they were. So we never went back to testing the accuracy fully. What could have been
#done was maybe K-fold or F1-score calculation to check if that provided more accurate score or if still overfitted.
metrics = trainer_bert.evaluate(test_dataset)
print(metrics)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
***** Running evaluation *****


{'accuracy': 1.0}


In [ ]:
#Here we start the labeling process on the filtered dataset that we got earlier. We predict the labels using our model, and then add them afterwards in a new column "predicted_label".
texts_to_label = filtered_ds["train"]["text"]
predicted_labels = []
for text in tqdm(texts_to_label, desc="Generating Predictions", unit="text"):
    predicted_label = model_bert.predict([text])[0]
    predicted_labels.append(predicted_label)

filtered_ds["train"] = filtered_ds["train"].add_column("predicted_label", predicted_labels)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Generating Predictions:   0%|          | 0/6892 [00:00<?, ?text/s]

Flattening the indices:   0%|          | 0/6892 [00:00<?, ? examples/s]

In [ ]:
#We check that the column was added
column_names = filtered_ds["train"].column_names
print("Column names:", column_names)



Column names: ['id', 'url', 'title', 'text', 'predicted_label']


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#We count the number of relevant rows. In case this not rendered when handing in, it was 1223 rows of data.
relevant_count = sum(1 for label in filtered_ds["train"]["predicted_label"] if label == "relevant")
print(f"Number of rows with predicted_label 'relevant': {relevant_count}")



Number of rows with predicted_label 'relevant': 1223


In [ ]:
#We export the data of the rows labeled relevant to CSV so we can continue working in the next notebook with this newly acquired dataset.
relevant_rows = filtered_ds["train"].filter(lambda example: example["predicted_label"] == "relevant")
relevant_df = pd.DataFrame(relevant_rows)
relevant_df.to_csv("relevant_data.csv", index=False)

Relevant data has been exported to 'relevant_data.csv'.
